In [9]:
# Import the required libraries.
import os
import cv2
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt
from threading import Thread
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

In [10]:
# Specify the height and width to which each video frame will be resized in our dataset.
IMAGE_HEIGHT , IMAGE_WIDTH = 64, 64

# Specify the number of frames of a video that will be fed to the model as one sequence.
SEQUENCE_LENGTH = 10

# Specify the directory containing the UCF50 dataset. 
DATASET_DIR = "dataset"

# Specify the list containing the names of the classes used for training. Feel free to choose any set of classes.
CLASSES_LIST = ["0", "1", "2", "3"]

In [11]:
def create_LRCN_model():
    '''
    This function will construct the required LRCN model.
    Returns:
        model: It is the required constructed LRCN model.
    '''

    # We will use a Sequential model for model construction.
    model = Sequential()
    
    # Define the Model Architecture.
    ########################################################################################################################
    
    model.add(TimeDistributed(Conv2D(16, (3, 3), padding='same',activation = 'relu'),
                              input_shape = (SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
    model.add(BatchNormalization())
    model.add(TimeDistributed(MaxPooling2D((4, 4)))) 
    model.add(TimeDistributed(Dropout(0.25)))
    
    model.add(TimeDistributed(Conv2D(32, (3, 3), padding='same',activation = 'relu')))
    model.add(BatchNormalization())
    model.add(TimeDistributed(MaxPooling2D((4, 4))))
    model.add(TimeDistributed(Dropout(0.25)))
    
    model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',activation = 'relu')))
    model.add(BatchNormalization())
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Dropout(0.25)))
    
    model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',activation = 'relu')))
    model.add(BatchNormalization())
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Dropout(0.25)))
                                      
    model.add(TimeDistributed(Flatten()))
                                      
    model.add(LSTM(32))
    
    model.add(Dense(len(CLASSES_LIST), activation = 'softmax'))
    
    ########################################################################################################################

    # Display the models summary.
    model.summary()
    
    # Return the constructed LRCN model.
    return model

In [12]:
lrcn_model = create_LRCN_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 10, 64, 64, 16)    448       
_________________________________________________________________
batch_normalization (BatchNo (None, 10, 64, 64, 16)    64        
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 16, 16, 16)    0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 10, 16, 16, 16)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 10, 16, 16, 32)    4640      
_________________________________________________________________
batch_normalization_1 (Batch (None, 10, 16, 16, 32)    128       
_________________________________________________________________
time_distributed_4 (TimeDist (None, 10, 4, 4, 32)      0

Choose one of the pre-trained weights.
Difference is in datasets.

In [13]:
#weights_lrcn = "5LRCN_model___Date_Time_2022_05_26__11_11_27___Loss_0.648440957069397___Accuracy_0.7206477522850037.h5"
#weights_lrcn = "6LRCN_model_second_dataset_3deadrows_seq5.h5"
weights_lrcn = "6LRCN_model__double_data_seq5_3deadrows.h5"
#weights_lrcn = '7LRCN_model___Date_Time_2022_05_26__17_00_07___Loss_0.6486591696739197___Accuracy_0.7248986959457397.h5'
lrcn_model.load_weights(weights_lrcn)
print(f'Sanity check: {lrcn_model.predict(np.random.random((1,10,64,64,3)))} %')

Sanity check: [[6.9648826e-01 2.9688773e-01 4.1595515e-04 6.2080184e-03]] %


In [14]:
import ifxdaq
import processing
import numpy as np
#print(ifxdaq.__version__)
from ifxdaq.sensor.radar_ifx import RadarIfxAvian
import os

config_file = "radar_configs/RadarIfxBGT60.json"

In [15]:
## Run this to understand the current radar settings better
import json
with open(config_file) as json_file:
    c = json.load(json_file)["device_config"]["fmcw_single_shape"]
    chirp_duration = c["num_samples_per_chirp"]/c['sample_rate_Hz']
    frame_duration = (chirp_duration + c['chirp_repetition_time_s']) * c['num_chirps_per_frame']
    print("With the current configuration, the radar will send out " + str(c['num_chirps_per_frame']) + \
          ' signals with varying frequency ("chirps") between ' + str(c['start_frequency_Hz']/1e9) + " GHz and " + \
          str(c['end_frequency_Hz']/1e9) + " GHz.")
    print('Each chirp will consist of ' + str(c["num_samples_per_chirp"]) + ' ADC measurements of the IF signal ("samples").')
    print('A chirp takes ' + str(chirp_duration*1e6) + ' microseconds and the delay between the chirps is ' + str(c['chirp_repetition_time_s']*1e6) +' microseconds.')
    print('With a total frame duration of ' + str(frame_duration*1e3) + ' milliseconds and a delay of ' + str(c['frame_repetition_time_s']*1e3) + ' milliseconds between the frame we get a frame rate of ' + str(1/(frame_duration + c['frame_repetition_time_s'])) + ' radar frames per second.')

With the current configuration, the radar will send out 64 signals with varying frequency ("chirps") between 60.5 GHz and 61.5 GHz.
Each chirp will consist of 128 ADC measurements of the IF signal ("samples").
A chirp takes 64.0 microseconds and the delay between the chirps is 400.0 microseconds.
With a total frame duration of 29.696 milliseconds and a delay of 50.0 milliseconds between the frame we get a frame rate of 12.547681188516362 radar frames per second.


In [16]:
def CNN_LSTM(range_doppler_map):
        range_doppler_map[:,:,:,0] /= np.max(range_doppler_map[:,:,:,0])
        range_doppler_map[:,:,:,1] /= np.max(range_doppler_map[:,:,:,1])
        range_doppler_map[:,:,:,2] /= np.max(range_doppler_map[:,:,:,2])
        pred = lrcn_model.predict(range_doppler_map[None,...])
        print(np.argmax(pred))

### LRCN with aggregated data processing

In [ ]:
import time

data_buffer = np.zeros((1,5,3,64,128))
counter = 0
sequence_size = 5

number_of_frames=500

t1 = time.time()
with RadarIfxAvian(config_file) as device:                             # Initialize the radar with configurations
    
    for i_frame, frame in enumerate(device):                           # Loop through the frames coming from the radar
        
        raw_data_frame = np.squeeze(frame['radar'].data/(4095.0))      # Dividing by 4095.0 to scale the data
        
        data_buffer[0,counter,...] = raw_data_frame
        counter = (counter + 1) % sequence_size
        
        if i_frame < sequence_size: continue
        
        input_data = np.concatenate((data_buffer[0,counter:,...], data_buffer[0,:counter,...]))
        range_doppler_map = np.abs(processing.processing_rangeDopplerData(input_data).transpose((0,2,3,1)))
        range_doppler_map[:,31:34,:,:] = 0
        t = Thread(target=CNN_LSTM, args=(range_doppler_map/255,))
        t.start()
        t.join()
        
        if(i_frame == number_of_frames-1):
            break   
            
t2 = time.time()

print(f"Elapsed time: {t2-t1:.2f}s")
print(f"Frame rate: {number_of_frames/(t2-t1)}")
